# Logging, Tracking, and Debugging Prompts using Comet

In this section, we will demonstrate how to log, track, and debug prompt using the `comet-llm` library. `comet-llm` is an open-sourced repo managed by Comet. Please give the repo star if you have a chance and submit any feedback you have! https://github.com/comet-ml/comet-llm

Let's first load all the necessary libraries:


In [2]:
! pip install openai comet-llm --quiet

In [3]:
import openai
import os
import IPython
import json
import pandas as pd
import numpy as np
import comet_llm
import urllib

from dotenv import load_dotenv
from pathlib import Path

dotenv_path = Path('../.env')
load_dotenv(dotenv_path=dotenv_path) 

# API configuration
openai.api_key = os.getenv("OPENAI_API_KEY")
COMET_API_KEY = os.getenv("COMET_API_KEY")
COMET_WORKSPACE = os.getenv("COMET_WORKSPACE")

The function below helps to generate the final results from the model after calling the OpenAI API:

In [5]:
def get_completion(messages, model="gpt-3.5-turbo", temperature=0, max_tokens=300):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
    )
    return response.choices[0].message["content"]

### Load the Data

The code below loads both the few-shot demonstrations and the validation dataset used for testing the model.

In [6]:
# print markdown
def print_markdown(text):
    """Prints text as markdown"""
    IPython.display.display(IPython.display.Markdown(text))

# load validation data from GitHub
f = urllib.request.urlopen("https://raw.githubusercontent.com/comet-ml/comet-llmops/main/data/article-tags.json")
val_data = json.load(f)

# load few shot data from GitHub
f = urllib.request.urlopen("https://raw.githubusercontent.com/comet-ml/comet-llmops/main/data/few_shot.json")
few_shot_data = json.load(f)

The following is a helper function to obtain the final predictions from the model given a prompt template (e.g., zero-shot or few-shot) and the provided input data.

In [7]:
def get_predictions(prompt_template, inputs):

    responses = []

    for i in range(len(inputs)):
        messages = messages = [
            {
                "role": "system",
                "content": prompt_template.format(input=inputs[i])
            }
        ]
        response = get_completion(messages)
        responses.append(response)

    return responses

### Few-Shot

First, we define a few-shot template which will leverage the few-shot demonstration data loaded previously.

In [8]:
# function to define the few-shot template
def get_few_shot_template(few_shot_prefix, few_shot_suffix, few_shot_examples):
    return few_shot_prefix + "\n\n" + "\n".join([ "Abstract: "+ ex["abstract"] + "\n" + "Tags: " + str(ex["tags"]) + "\n" for ex in few_shot_examples]) + "\n\n" + few_shot_suffix

# function to sample few shot data
def random_sample_data (data, n):
    return np.random.choice(few_shot_data, n, replace=False)


# the few-shot prefix and suffix
few_shot_prefix = """Your task is to extract model names from machine learning paper abstracts. Your response is an an array of the model names in the format [\"model_name\"]. If you don't find model names in the abstract or you are not sure, return [\"NA\"]"""
few_shot_suffix = """Abstract: {input}\nTags:"""

# load 3 samples from few shot data
few_shot_template = get_few_shot_template(few_shot_prefix, few_shot_suffix, random_sample_data(few_shot_data, 3))

In [9]:
few_shot_template

'Your task is to extract model names from machine learning paper abstracts. Your response is an an array of the model names in the format ["model_name"]. If you don\'t find model names in the abstract or you are not sure, return ["NA"]\n\nAbstract: Prevalent semantic segmentation solutions are, in essence, a dense discriminative classifier of p(class|pixel feature). Though straightforward, this de facto paradigm neglects the underlying data distribution p(pixel feature|class), and struggles to identify out-of-distribution data. Going beyond this, we propose GMMSeg, a new family of segmentation models that rely on a dense generative classifier for the joint distribution p(pixel feature,class). For each class, GMMSeg builds Gaussian Mixture Models (GMMs) via Expectation-Maximization (EM), so as to capture class-conditional densities. Meanwhile, the deep dense representation is end-to-end trained in a discriminative manner, i.e., maximizing p(class|pixel feature). This endows GMMSeg with 

### Zero-Shot Template

The code below defines the zero-shot template. Note that we use the same instruction from the few-shot prompt template. But in this case, we don't use the demonstrations.

In [10]:
zero_shot_template = """
Your task is extract model names from machine learning paper abstracts. Your response is an an array of the model names in the format [\"model_name\"]. If you don't find model names in the abstract or you are not sure, return [\"NA\"]

Abstract: {input}
Tags:
"""

In [17]:
val_data[0]

{'abstract': 'Training large language models (LLM) with open-domain instruction following data brings colossal success. However, manually creating such instruction data is very time-consuming and labor-intensive. Moreover, humans may struggle to produce high-complexity instructions. In this paper, we show an avenue for creating large amounts of instruction data with varying levels of complexity using LLM instead of humans. Starting with an initial set of instructions, we use our proposed Evol-Instruct to rewrite them step by step into more complex instructions. Then, we mix all generated instruction data to fine-tune LLaMA. We call the resulting model WizardLM. Human evaluations on a complexity-balanced test bed show that instructions from Evol-Instruct are superior to human-created ones. By analyzing the human evaluation results of the high complexity part, we demonstrate that outputs from our WizardLM model are preferred to outputs from OpenAI ChatGPT. Even though WizardLM still lags

### Get Predictions

We then generated all the predictions using the validation data as inputs:

In [ ]:
# get the predictions

abstracts = [val_data[i]["abstract"] for i in range(len(val_data[:3]))]
few_shot_predictions = get_predictions(few_shot_template, abstracts)

In [19]:
zero_shot_predictions = get_predictions(zero_shot_template, abstracts)
expected_tags = [str(val_data[i]["tags"]) for i in range(len(val_data[:3]))]

In [20]:
print("Few shot predictions")
print(few_shot_predictions)
print("\n\nZero shot predictions")
print(zero_shot_predictions)
print("\n\nExpected tags")
print(expected_tags)

Few shot predictions
["['Evol-Instruct', 'LLaMA', 'WizardLM', 'ChatGPT']", "['FLAN-T5', 'AMR', 'UD', 'SRL', 'AMR2.0', 'AMR3.0', 'BioAMR', 'LoRA']", '["NA"]']


Zero shot predictions
['["WizardLM", "Evol-Instruct", "LLaMA", "ChatGPT"]', '["FLAN-T5"]', '["large language models", "generative AI", "hypothesis machines"]']


Expected tags
["['LLaMA', 'ChatGPT', 'WizardLM']", "['FLAN-T5', 'FLAN']", "['NA']"]


### Log Prompt Results

Finally, we log the prompt + results to Comet. Note that we are logging both the few-shot and zero-shot results, together with all the metadata and tags.

In [21]:
# log the predictions in Comet along with the ground truth for comparison

# initialize comet
comet_llm.init(COMET_API_KEY, COMET_WORKSPACE, project="ml-paper-tagger-prompts")

# log the predictions
for i in range(len(expected_tags)):
    # log the few-shot predictions
    comet_llm.log_prompt(
        prompt=few_shot_template.format(input=abstracts[i]),
        prompt_template=few_shot_template,
        output=few_shot_predictions[i],
        tags = ["gpt-3.5-turbo", "few-shot"],
        metadata = {
            "expected_tags": expected_tags[i],
            "abstract": abstracts[i],
        }
    )

    # log the zero-shot predictions
    comet_llm.log_prompt(
        prompt=zero_shot_template.format(input=abstracts[i]),
        prompt_template=zero_shot_template,
        output=zero_shot_predictions[i],
        tags = ["gpt-3.5-turbo", "zero-shot"],
        metadata = {
            "expected_tags": expected_tags[i],
            "abstract": abstracts[i],
        }
    )

COMET INFO: Valid Comet API Key saved in /root/.comet.config (set COMET_CONFIG to change where it is saved).


Prompt logged to https://www.comet.com/danselem/ml-paper-tagger-prompts


INFO:comet_llm.summary:Prompt logged to https://www.comet.com/danselem/ml-paper-tagger-prompts
